In [40]:
import requests
import urllib.parse
import json

def search_item(keyword):
    # 關鍵字做 URL 編碼
    kw = urllib.parse.quote(keyword)

    api_url = f"https://ecshweb.pchome.com.tw/search/v3.3/all/results?q={kw}&page=1&sort=rnk/dc"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    r = requests.get(api_url, headers=headers, timeout=10)
    r.raise_for_status()
    data = r.json()

    prods = data.get("prods") or []
    if not prods:
        return "無此商品"

    results = []

    for item in prods[:25]:
        name = item.get("name", "無資料")
        price = item.get("price", "無資料")

        product_id = item.get("Id", "")
        url = f"https://24h.pchome.com.tw/prod/{product_id}" if product_id else ""

        # 圖片路徑：用 picB / picS，前綴改成 cs-a.ecimg.tw
        pic_path = item.get("picB") or item.get("picS") or ""
        img = f"https://cs-a.ecimg.tw{pic_path}" if pic_path else ""

        results.append({
            "name": name,
            "price": price,
            "url": url,
            "img": img
        })

    # 存成 JSON 檔
    filename = f"{keyword}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    return results


# 測試
data = search_item("牛奶")
if isinstance(data, str):
    print(data)
else:
    for i, item in enumerate(data, 1):
        print(f"第 {i} 筆：")
        print(f"  名稱：{item['name']}")
        print(f"  價格：{item['price']}")
        print(f"  商品：{item['url']}")
        print(f"  圖片：{item['img']}")
        print("-" * 40)


第 1 筆：
  名稱：成份無調整保久乳-全脂牛乳200ml(24入/箱)x2
  價格：899
  商品：https://24h.pchome.com.tw/prod/DBAB08-1900I8G9J
  圖片：https://cs-a.ecimg.tw/items/DBAB081900I8G9J/000001_1760492594.jpg
----------------------------------------
第 2 筆：
  名稱：【南紡購物中心】 紐西蘭AnchorSGS認證1公升100%純牛奶保久乳(1Lx3瓶組合) 
  價格：459
  商品：https://24h.pchome.com.tw/prod/DBDA0G-A900AZ32U
  圖片：https://cs-a.ecimg.tw/items/DBDA0GA900AZ32U/000001_1656407706.jpg
----------------------------------------
第 3 筆：
  名稱：高鈣牛乳-無加糖200ml 24入
  價格：312
  商品：https://24h.pchome.com.tw/prod/DBAB7J-A90090XUI
  圖片：https://cs-a.ecimg.tw/items/DBAB7JA90090XUI/000001_1661614703.jpg
----------------------------------------
第 4 筆：
  名稱：保久乳-低脂高鈣牛乳200ml(24入)
  價格：269
  商品：https://24h.pchome.com.tw/prod/DBAB08-A49138250
  圖片：https://cs-a.ecimg.tw/items/DBAB08A49138250/000001_1545812747.jpg
----------------------------------------
第 5 筆：
  名稱：W PROTEIN JUNIOR成長期乳清蛋白飲-巧克力牛奶(35g*10包/盒)
  價格：320
  商品：https://24h.pchome.com.tw/prod/DFAX00-A900HB4NO
  圖片：https://cs-a.ecimg.tw

In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json   

def search_pchome(keyword: str, max_items: int = 10):
    """
    Use Selenium to search PChome.
    Returns:
      - If found: list[dict], each {name, price, img, link}
      - If nothing: "無此商品"
    """
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--incognito")
    options.add_argument("--disable-popup-blocking")

    driver = webdriver.Chrome(options=options)
    CARD = ".c-prodInfoV2.c-prodInfoV2--gridCard"

    try:
        # Go to search result page
        driver.get(f"https://24h.pchome.com.tw/search/?q={keyword}")
        time.sleep(3)

        # First batch of product cards
        items = driver.find_elements(By.CSS_SELECTOR, CARD)
        if not items:
            return "無此商品"

        last = items[-1]

        # Scroll down until no new items or enough items
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            new_items = driver.find_elements(By.CSS_SELECTOR, CARD)
            if not new_items or new_items[-1] == last or len(new_items) >= max_items * 3:
                items = new_items
                break

            items = new_items
            last = items[-1]

        results = []

        for card in items:
            # Scroll this card into view to trigger lazy-load image
            driver.execute_script("arguments[0].scrollIntoView();", card)
            time.sleep(0.2)

            # Name
            name = card.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__title").text
            if keyword not in name:
                continue

            # Price
            price = card.find_element(
                By.CSS_SELECTOR,
                ".c-prodInfoV2__priceValue.c-prodInfoV2__priceValue--m"
            ).text

            # Image (handle mobile_loading.svg lazy-load)
            img_el = card.find_element(By.CSS_SELECTOR, ".c-prodInfoV2__img img")
            img = img_el.get_attribute("src") or ""
            if "mobile_loading.svg" in img:
                for attr in ("data-src", "data-srcset", "srcset"):
                    v = img_el.get_attribute(attr)
                    if v:
                        img = v.split(",")[0].strip().split(" ")[0]
                        break
            if not img:
                img = "無圖片"

            # Product link
            link = card.find_element(
                By.CSS_SELECTOR,
                "a.c-prodInfoV2__link.gtmClickV2"
            ).get_attribute("href")

            results.append({
                "name": name,
                "price": price,
                "img": img,
                "link": link
            })

            if len(results) >= max_items:
                break

        # ---- ★ Save to JSON here ★ ----
        if results:
            filename = f"{keyword}.json"   # e.g. "衛生紙.json"
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=4)
            return results
        else:
            return "無此商品"

    finally:
        driver.quit()


if __name__ == "__main__":
    kw = "找不到的輸出結果"
    data = search_pchome(kw, max_items=50)

    if isinstance(data, str):
        # e.g. "無此商品"
        print(f"關鍵字「{kw}」搜尋結果：{data}")
    else:
        print(f"關鍵字「{kw}」找到 {len(data)} 筆商品：\n")
        for i, item in enumerate(data, start=1):
            print(f"第 {i} 筆")
            print(f"  名稱：{item['name']}")
            print(f"  價格：{item['price']}")
            print(f"  連結：{item['link']}")
            print(f"  圖片：{item['img']}")
            print("-" * 50)


關鍵字「找不到的輸出結果」搜尋結果：無此商品


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, json

#  第一階段：抓 PChome 搜尋頁所有商品資料（含可能為 null 的圖片）

def crawl_search_list(keyword):

    print("\n=====【階段 1：抓搜尋頁所有商品】=====\n")

    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)

    results = []

    for page in range(1, 200):
        print(f"=== 抓第 {page} 頁 ===")

        url = f"https://24h.pchome.com.tw/search/?q={keyword}&p={page}"
        driver.get(url)

        # 等待商品載入
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.CSS_SELECTOR,
                    ".c-prodInfoV2__title, .c-product__name"
                ))
            )
        except:
            print("頁面無商品 → 停止")
            break

        time.sleep(1)

        # 抓兩種 layout
        cards1 = driver.find_elements(By.CSS_SELECTOR, ".c-prodInfoV2")
        cards2 = driver.find_elements(By.CSS_SELECTOR, ".c-product")
        cards = cards1 + cards2

        print("本頁商品數：", len(cards))
        is_last_page = len(cards) < 40  

        # 將每個卡片的基本資料抓起來
        for card in cards:

            # 強制滾動確保資料顯示
            driver.execute_script("arguments[0].scrollIntoView();", card)
            time.sleep(0.15)

            # 名稱
            try:
                name = card.find_element(
                    By.CSS_SELECTOR,
                    ".c-prodInfoV2__title, .c-product__name"
                ).text.strip()
            except:
                name = ""

            # 價格
            try:
                price = card.find_element(
                    By.CSS_SELECTOR,
                    ".c-prodInfoV2__priceValue--m, .c-product__price"
                ).text.strip()
            except:
                price = ""

            # 連結
            try:
                link = card.find_element(
                    By.CSS_SELECTOR,
                    "a.c-prodInfoV2__link, a.c-product__img, a"
                ).get_attribute("href")
            except:
                link = ""

            # 圖片（列表頁先抓，有可能為 null）
            img = None
            try:
                container = card.find_element(
                    By.CSS_SELECTOR,
                    ".c-prodInfoV2__img, .c-product__img"
                )

                # <img> 標籤
                try:
                    img_tag = container.find_element(By.TAG_NAME, "img")
                    img = img_tag.get_attribute("src")
                    if img and "loading.svg" in img:
                        img = img_tag.get_attribute("data-src")
                except:
                    pass

                # 背景圖片
                if not img:
                    style = container.get_attribute("style")
                    if "background-image" in style:
                        img = style.split("url(")[1].split(")")[0].strip('"').strip("'")

            except:
                img = None

            results.append({
                "name": name,
                "price": price,
                "link": link,
                "img": img
            })

        if is_last_page:
            print("→ 已到最後一頁，停止")
            break

        time.sleep(1)

    driver.quit()
    print(f"\n✔ 階段 1 完成，共 {len(results)} 筆商品\n")
    return results

#  第二階段：補上所有 null 圖片（進入商品頁抓封面）

def fill_cover_images(results):

    print("\n=====【階段 2：補圖片（抓商品頁封面）】=====\n")

    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)

    for item in results:

        if item["img"]:   # 若列表頁已有圖片 → 跳過
            continue

        link = item["link"]
        if not link:
            continue

        print(f"→ 列表無圖片，進入商品頁補抓：{link}")

        try:
            driver.get(link)

            WebDriverWait(driver, 8).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".pic-main img"))
            )
            time.sleep(0.4)

            cover = driver.find_element(By.CSS_SELECTOR, ".pic-main img").get_attribute("src")
            item["img"] = cover

        except:
            print("   ❌ 補抓失敗")
            item["img"] = None

    driver.quit()
    print("\n✔ 階段 2 完成（圖片補抓結束）\n")
    return results

def search_pchome_final(keyword):

    data = crawl_search_list(keyword)
    data = fill_cover_images(data)

    filename = f"{keyword}_final.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"\n🎉【全部完成！】")
    print(f"✔ 最終筆數：{len(data)}")
    print(f"✔ 已輸出：{filename}")

    return data

search_pchome_final("衛生紙")



=====【階段 1：抓搜尋頁所有商品】=====

=== 抓第 1 頁 ===
本頁商品數： 40
=== 抓第 2 頁 ===
本頁商品數： 40
=== 抓第 3 頁 ===
本頁商品數： 40
=== 抓第 4 頁 ===
本頁商品數： 40
=== 抓第 5 頁 ===
本頁商品數： 40
=== 抓第 6 頁 ===
本頁商品數： 40
=== 抓第 7 頁 ===
本頁商品數： 40
=== 抓第 8 頁 ===
本頁商品數： 40
=== 抓第 9 頁 ===
本頁商品數： 40
=== 抓第 10 頁 ===
本頁商品數： 40
=== 抓第 11 頁 ===
本頁商品數： 40
=== 抓第 12 頁 ===
本頁商品數： 40
=== 抓第 13 頁 ===
本頁商品數： 40
=== 抓第 14 頁 ===
本頁商品數： 40
=== 抓第 15 頁 ===
本頁商品數： 40
=== 抓第 16 頁 ===
本頁商品數： 40
=== 抓第 17 頁 ===
本頁商品數： 40
=== 抓第 18 頁 ===
本頁商品數： 40
=== 抓第 19 頁 ===
本頁商品數： 40
=== 抓第 20 頁 ===
本頁商品數： 40
=== 抓第 21 頁 ===
本頁商品數： 40
=== 抓第 22 頁 ===
本頁商品數： 40
=== 抓第 23 頁 ===
本頁商品數： 40
=== 抓第 24 頁 ===
本頁商品數： 40
=== 抓第 25 頁 ===
本頁商品數： 40
=== 抓第 26 頁 ===
本頁商品數： 40
=== 抓第 27 頁 ===
本頁商品數： 40
=== 抓第 28 頁 ===
本頁商品數： 40
=== 抓第 29 頁 ===
本頁商品數： 40
=== 抓第 30 頁 ===
本頁商品數： 40
=== 抓第 31 頁 ===
本頁商品數： 40
=== 抓第 32 頁 ===
本頁商品數： 40
=== 抓第 33 頁 ===
本頁商品數： 40
=== 抓第 34 頁 ===
本頁商品數： 40
=== 抓第 35 頁 ===
本頁商品數： 40
=== 抓第 36 頁 ===
本頁商品數： 40
=== 抓第 37 頁 ===
本頁商品數： 40
=== 抓第 38 頁 ===
本頁商

[{'name': 'Kenji 健司 香蔥蘇打餅乾(300g/20包)',
  'price': '$119',
  'link': 'https://24h.pchome.com.tw/prod/DBAR5U-A9008P1MC',
  'img': 'https://img.pchome.com.tw/cs/items/DBAR5UA9008P1MC/000001_1698638447.jpg?width=480'},
 {'name': '義美 純麥取向蘇打餅乾(袋裝)341g',
  'price': '$68',
  'link': 'https://24h.pchome.com.tw/prod/DBBP0G-A900B3CHA',
  'img': 'https://img.pchome.com.tw/cs/items/DBBP0GA900B3CHA/000007_1719542545.jpg?width=480'},
 {'name': '華元 餅乾派對箱504g/箱(48包入)',
  'price': '$299',
  'link': 'https://24h.pchome.com.tw/prod/DBAC90-A900G46YD',
  'img': 'https://img.pchome.com.tw/cs/items/DBAC90A900G46YD/000001_1753261130.jpg?width=480'},
 {'name': 'CHEETOS 奇多 隨口脆336g系列 x1組 〔 家常起司 雞汁 海苔 玉米濃湯 2倍濃起司 〕',
  'price': '$180',
  'link': 'https://24h.pchome.com.tw/prod/DBAC02-A900HF4YF',
  'img': 'https://img.pchome.com.tw/cs/items/DBAC02A900HF4YF/000007_1741656107.jpg?width=480'},
 {'name': '義美 起司取向蘇打餅乾(袋裝)300g',
  'price': '$68',
  'link': 'https://24h.pchome.com.tw/prod/DBBP0G-A900B3CJC',
  'img': 'https

In [7]:
import json

with open("餅乾_final.json", "r", encoding="utf-8") as f:
    data = json.load(f)

total = len(data)
null_img = sum(1 for item in data if not item["img"])

print("總筆數：", total)
print("img 為 null 的筆數：", null_img)
print("比例：", null_img / total if total else 0)


總筆數： 4010
img 為 null 的筆數： 2
比例： 0.0004987531172069825


In [8]:
import json

with open("餅乾_final.json", "r", encoding="utf-8") as f:
    data = json.load(f)

total = len(data)

null_indices = []  # 用來記錄哪些筆的 img 是 null

for i, item in enumerate(data):
    img = item.get("img")
    if not img:  # None、""、0 都會被視為 False，這邊對我們剛好
        null_indices.append(i)

print("總筆數：", total)
print("img 為 null 的筆數：", len(null_indices))

# 印出是哪幾筆（第幾行）
for idx in null_indices:
    # idx 是 0-based，第幾筆我們用 1-based 比較直覺
    print(f"第 {idx + 1} 筆是 null：")
    print("  名稱：", data[idx].get("name"))
    print("  連結：", data[idx].get("link"))
    print("  圖片：", data[idx].get("img"))
    print("-" * 40)


總筆數： 4010
img 為 null 的筆數： 2
第 3392 筆是 null：
  名稱： MUJI 無印良品 鹽味餅乾80g
  連結： 
  圖片： None
----------------------------------------
第 3661 筆是 null：
  名稱： MUJI 無印良品 米果棒 抹茶可可33g
  連結： 
  圖片： None
----------------------------------------


In [1]:
import requests
import json
import time

# 建議自己帶 User-Agent，模擬正常瀏覽器
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

# PChome 搜尋 API 的固定 base URL
API_URL = "https://ecshweb.pchome.com.tw/search/v3.3/all/results"


def fetch_pchome_page(keyword: str, page: int) -> list:
    """
    打 PChome 的搜尋 API，拿「某一頁」的商品列表。
    回傳一個 list，每個元素是原始 API 回傳的商品 dict。
    """
    params = {
        "q": keyword,      # 搜尋關鍵字（requests 會自動 urlencode）
        "page": page,      # 第幾頁（API 一頁 20 筆）
        "sort": "rnk/dc",  # 排序方式（照你 DevTools 上看到的來）
    }

    resp = requests.get(API_URL, params=params, headers=HEADERS, timeout=10)
    resp.raise_for_status()

    data = resp.json()
    items = data.get("prods", [])  # 商品列表都在 "prods" 裡

    return items


def crawl_pchome_requests(keyword: str,
                          max_pages: int = 200,
                          sleep_sec: float = 0.2) -> list:
    """
    連續打很多頁的搜尋 API，把所有商品攤平成一個 list。
    max_pages：最多抓幾頁（保險上限）
    sleep_sec：兩頁之間稍微睡一下，避免打太兇
    """
    all_products = []

    for page in range(1, max_pages + 1):
        print(f"=== 抓第 {page} 頁 ===")

        try:
            items = fetch_pchome_page(keyword, page)
        except requests.RequestException as e:
            print(f"第 {page} 頁請求失敗：{e}")
            break

        print("本頁商品數：", len(items))

        # 這一頁完全沒有資料了 → 視為結束
        if not items:
            print("→ 沒有更多商品，停止")
            break

        for p in items:
            # 1️⃣ 先拿原始圖片路徑（通常是相對路徑 /items/...）
            raw_img = p.get("picB") or p.get("picS") or p.get("img") or ""

            img_url = None
            if raw_img:
                if raw_img.startswith("http"):
                    # 後端如果已經給完整網址，就直接用
                    img_url = raw_img
                else:
                    # 否則當成相對路徑，加上固定前綴
                    # 你給的範例是：
                    #   /items/DEAG7PA900B0BNY/000001_1679706993.jpg
                    #   -> https://img.pchome.com.tw/cs/items/DEAG7PA900B0BNY/000001_1679706993.jpg
                    img_url = "https://img.pchome.com.tw/cs" + raw_img

            all_products.append({
                "name": p.get("name", ""),
                "price": p.get("price", 0),
                "link": f"https://24h.pchome.com.tw/prod/{p.get('Id')}",
                "img": img_url,
            })

        # 稍微睡一下，友善一點
        time.sleep(sleep_sec)

    print(f"\n✔ 總共抓到 {len(all_products)} 筆商品\n")
    return all_products


def search_pchome_final_requests(keyword: str):
    """
    封裝好的入口：
    - 抓所有頁面
    - 寫成 <keyword>_requests_final.json
    """
    data = crawl_pchome_requests(keyword, max_pages=200, sleep_sec=0.2)

    filename = f"{keyword}_requests_final.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print("🎉 全部完成！")
    print(f"✔ 最終筆數：{len(data)}")
    print(f"✔ 已輸出：{filename}")

    return data


if __name__ == "__main__":
    # 這裡可以改成你想搜尋的任何關鍵字
    search_pchome_final_requests("紅酒")


=== 抓第 1 頁 ===
本頁商品數： 20
=== 抓第 2 頁 ===
本頁商品數： 20
=== 抓第 3 頁 ===
本頁商品數： 20
=== 抓第 4 頁 ===
本頁商品數： 20
=== 抓第 5 頁 ===
本頁商品數： 20
=== 抓第 6 頁 ===
本頁商品數： 20
=== 抓第 7 頁 ===
本頁商品數： 20
=== 抓第 8 頁 ===
本頁商品數： 20
=== 抓第 9 頁 ===
本頁商品數： 20
=== 抓第 10 頁 ===
本頁商品數： 20
=== 抓第 11 頁 ===
本頁商品數： 20
=== 抓第 12 頁 ===
本頁商品數： 20
=== 抓第 13 頁 ===
本頁商品數： 20
=== 抓第 14 頁 ===
本頁商品數： 20
=== 抓第 15 頁 ===
本頁商品數： 20
=== 抓第 16 頁 ===
本頁商品數： 20
=== 抓第 17 頁 ===
本頁商品數： 20
=== 抓第 18 頁 ===
本頁商品數： 20
=== 抓第 19 頁 ===
本頁商品數： 20
=== 抓第 20 頁 ===
本頁商品數： 20
=== 抓第 21 頁 ===
本頁商品數： 20
=== 抓第 22 頁 ===
本頁商品數： 20
=== 抓第 23 頁 ===
本頁商品數： 20
=== 抓第 24 頁 ===
本頁商品數： 20
=== 抓第 25 頁 ===
本頁商品數： 20
=== 抓第 26 頁 ===
本頁商品數： 20
=== 抓第 27 頁 ===
本頁商品數： 20
=== 抓第 28 頁 ===
本頁商品數： 20
=== 抓第 29 頁 ===
本頁商品數： 20
=== 抓第 30 頁 ===
本頁商品數： 20
=== 抓第 31 頁 ===
本頁商品數： 20
=== 抓第 32 頁 ===
本頁商品數： 20
=== 抓第 33 頁 ===
本頁商品數： 20
=== 抓第 34 頁 ===
本頁商品數： 20
=== 抓第 35 頁 ===
本頁商品數： 20
=== 抓第 36 頁 ===
本頁商品數： 20
=== 抓第 37 頁 ===
本頁商品數： 20
=== 抓第 38 頁 ===
本頁商品數： 20
=== 抓第 39 頁 ===
本頁商品數

In [2]:
import json

with open("紅酒_requests_final.json", "r", encoding="utf-8") as f:
    data = json.load(f)

total = len(data)
null_img = sum(1 for item in data if not item["img"])

print("總筆數：", total)
print("img 為 null 的筆數：", null_img)
print("比例：", null_img / total if total else 0)


總筆數： 1744
img 為 null 的筆數： 0
比例： 0.0
